In [ ]:
import pandas as pd


# Importing the Dataframes

In [ ]:
# load the CSV file products from the directory data

products = pd.read_csv("products.csv")

In [ ]:
# load the CSV file orders from the directory data

orders = pd.read_csv("orders.csv")

In [ ]:
# load the CSV file orderlines from the directory data 

orderlines = pd.read_csv("orderlines.csv")

In [ ]:
# load the CSV file brands from the directory data 
brands = pd.read_csv("brands.csv")

# Exploration Order_lines

In [ ]:
orderlines.head()


,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38


In [ ]:
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


In [ ]:
orderlines.product_quantity.min()
# no lines with zero products

In [ ]:
# Check for Duplicates
orderlines.drop_duplicates().shape == orderlines.shape
orderlines["id"].drop_duplicates().shape == orderlines["id"].shape

In [ ]:
# Amount of duplicate order_ids in order_line dataframe
len(orderlines["id_order"])-len(orderlines["id_order"].drop_duplicates())

89128

In [ ]:
orderlines.loc[orderlines.duplicated(subset = "id_order"),:]

,id,id_order,product_id,product_quantity,sku,unit_price,date
14,1119130,299552,0,1,SAT0044,19.99,2017-01-01 02:14:36
22,1119143,299559,0,1,APP1214,84.99,2017-01-01 02:36:43
45,1119198,299583,0,1,BEL0275,16.99,2017-01-01 08:58:33
46,1119199,299583,0,1,BEL0286,17.99,2017-01-01 09:00:26
61,1119216,299596,0,1,PAC1044,2.780.99,2017-01-01 09:55:45
...,...,...,...,...,...,...,...
293944,1650149,527367,0,1,REP0299,59.99,2018-03-14 13:40:04
293948,1650153,527369,0,1,APP0927,13.99,2018-03-14 13:40:39
293954,1650162,527369,0,1,BEL0307,27.99,2018-03-14 13:42:41
293957,1650168,527377,0,1,APP2501,391.00,2018-03-14 13:44:27


# CLEANING ORDER_LINES

In [ ]:
orderlines_cl = orderlines.copy()

In [ ]:
# Change the date column to Datetime datatype
orderlines_cl["date"]= pd.to_datetime(orderlines_cl["date"])

Some numbers in the unit_price column are corrupted. There are 2 dots in between numbers. The format needs to be corrected to convert the column from string to float format

In [ ]:
# find the lines with corrupted prices
orderlines_cl.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()

False    257814
True      36169
Name: unit_price, dtype: int64

In [ ]:
# Share of prices
percentage = ((orderlines_cl.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()[1]/orderlines_cl.shape[0]))
percentage

0.12303092355680431

In [ ]:
#separate the corrupted prices
twodot_prices = orderlines_cl.loc[orderlines_cl.unit_price.str.contains("\d+\.\d+\.\d+"),:].copy()

In [ ]:
#correct prices only
onedot_prices = orderlines_cl.loc[orderlines_cl.unit_price.str.contains("\d+\.\d+\.\d+")==False,:].copy()

In [ ]:
# remove the first . in the prices
twodot_prices["unit_price"] = twodot_prices.unit_price.str.replace(r'(\d+).(\d+).(\d+)', r'\1\2.\3', regex=True)

In [ ]:
#create a numeric value
twodot_prices["unit_price"] = pd.to_numeric(twodot_prices["unit_price"])

In [ ]:
#create a numeric value
onedot_prices["unit_price"] = pd.to_numeric(onedot_prices["unit_price"])

In [ ]:
#Merge the two parts together again
all_dots = [twodot_prices,onedot_prices]
orderlines_cl = pd.concat(all_dots)

In [ ]:
#Remove outlier of 159,989
orderlines_cl["unit_price"] = orderlines_cl.loc[orderlines_cl["unit_price"].between(0,16000),"unit_price"]

In [ ]:
#remove product_id column
orderlines_cl.drop("product_id", axis = 1)

In [ ]:
orderlines_cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293983 entries, 6 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293981 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 17.9+ MB


In [ ]:
# 2 Values are Null Values in unit_price
orderlines_cl.loc[orderlines_cl["unit_price"].isna(),:]

,id,id_order,product_id,product_quantity,sku,unit_price,date
36979,1197439,331780,0,1,NEA0009,NaN,2017-03-03 22:29:21
77008,1268645,365886,0,1,APP1465,NaN,2017-06-15 12:48:54


In [ ]:
# remove the Null Values in Unit Price
orderlines_cl = orderlines_cl.loc[~(orderlines_cl["unit_price"].isna()),:]

#Clean Orders


In [ ]:
orders.head()

,order_id,created_date,total_paid,state
0,241319,2017-01-02 13:35:40,44.99,Cancelled
1,241423,2017-11-06 13:10:02,136.15,Completed
2,242832,2017-12-31 17:40:03,15.76,Completed
3,243330,2017-02-16 10:59:38,84.98,Completed
4,243784,2017-11-24 13:35:19,157.86,Cancelled


In [ ]:
#create a date type for created_date
orders_cl = orders.copy()
orders_cl["created_date"] = pd.to_datetime(orders_cl["created_date"])

In [ ]:
# delete the 5 null values from total paid
orders_cl = orders_cl.loc[~(orders_cl["total_paid"].isna()),:]

In [ ]:
orders_cl.head()

,order_id,created_date,total_paid,state
0,241319,2017-01-02 13:35:40,44.99,Cancelled
1,241423,2017-11-06 13:10:02,136.15,Completed
2,242832,2017-12-31 17:40:03,15.76,Completed
3,243330,2017-02-16 10:59:38,84.98,Completed
4,243784,2017-11-24 13:35:19,157.86,Cancelled


In [ ]:
#Download clean orders CSV
orders_cl.to_csv("orders_clean.csv",index=False)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

orderlines_cl.to_csv("orderlines_clean.csv", index=False)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>